# BERT for BETO

In [315]:
import pandas as pd
import numpy as np
import torch
import random
import os
import nltk
from nltk import tokenize

In [136]:
#Load in some data
save_path = '/Users/Jonathan/Desktop/LabeledChemEData/Labeled_Sheets/'
#Do we have something that allows us to fill empty cells with "" or something else? Maybe set any NaNs to 0? That would imply
#A null label. 
df = pd.read_excel(save_path+"Carbon_0.xlsx")

In [50]:
df

,Unnamed: 0,name,tokens,BESIO,entity,mol_class,Unnamed: 6,name.1,tokens.1,BESIO.1,...,tokens.48,BESIO.48,entity.48,mol_class.48,Unnamed: 294,name.49,tokens.49,BESIO.49,entity.49,mol_class.49
0,0,Jon O,In,NaN,NaN,NaN,0,Jon O,X-ray,NaN,...,©,NaN,NaN,NaN,0,Jon O,©,NaN,NaN,NaN
1,1,2010,the,NaN,NaN,NaN,1,2014,photoelectron,NaN,...,2020,NaN,NaN,NaN,1,2015,2015,NaN,NaN,NaN
2,2,250,interaction,NaN,NaN,NaN,2,114,spectroscopy,NaN,...,Elsevier,NaN,NaN,NaN,2,104,Elsevier,NaN,NaN,NaN
3,3,10.1016/j.carbon.2010.02.003,between,NaN,NaN,NaN,3,10.1016/j.carbon.2013.12.061,(XPS),NaN,...,Ltd,NaN,NaN,NaN,3,10.1016/j.carbon.2015.08.007,Ltd.,NaN,NaN,NaN
4,4,NaN,gas,NaN,NaN,NaN,4,NaN,has,NaN,...,The,NaN,NaN,NaN,4,NaN,Hydrogels,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,349,NaN,NaN,NaN,NaN,NaN,349,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,349,NaN,NaN,NaN,NaN,NaN
350,350,NaN,NaN,NaN,NaN,NaN,350,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,350,NaN,NaN,NaN,NaN,NaN
351,351,NaN,NaN,NaN,NaN,NaN,351,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,351,NaN,NaN,NaN,NaN,NaN
352,352,NaN,NaN,NaN,NaN,NaN,352,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,352,NaN,NaN,NaN,NaN,NaN


Need to update extract_xy_ to pull out just the x values (ie, tokens), and the y values as a smashed together BESIO. He used NLTK sent_tokenize, so we are in the same place, and we can just go back to having sentences or something.

In [465]:
def extract_xy_(df):
    """
    This method extracts and correctly aranges the NER training x-values (tokens)
    and y-values (BESIO labels) from a pandas dataframe containing labeled NER
    data

    Parameters:
        df (pandas DataFrame, required): Dataframe loaded via pd.read_excel() on
            a labeled NER dataset

        endings_dict (dictionary, required): Dictionary containing the indicies
            where each sentence in each line of tokens ends.

    Returns:
        two lists of identical shape. One contains all the words that were tokenized for labeling
        and the other contains all the labels of those tokenized words.
    """
    labeled = []
    columns = df.columns
    new_df = pd.DataFrame()
    all_tokens = []
    besio = []
    mol = []
    IorO = []
        
    for idx, column in enumerate(columns):
        # find every column that starts with 'name'
        if column.startswith('name'):

            # check if the entry in 'name' cell is a str
            if isinstance(df[column][0], str):
                tokens = df[columns[idx + 1]].values
                #find the index where the tokens become NaNs, and chop the token length down to that size. 
                l = 0
                for entries in tokens: 
                    if type(entries) == str:
                        l += 1
                all_tokens.append(tokens[:l])
                df[columns[idx+2]].replace(np.nan, 'O', inplace = True)
                besio.append(df[columns[idx+2]][:l].values)
                df[columns[idx+3]].replace(np.nan, '', inplace = True)
                mol.append(df[columns[idx+3]][:l].values)
                df[columns[idx+4]].replace(np.nan, '', inplace = True)
                IorO.append(df[columns[idx+4]][:l].values)

    i = 0
    label_values = []
    while i < len(besio):
        label_values.append([])
        for j in range(len(besio[i])):
            #Strip unintentional whitespace from all entries:
            besio[i][j] = besio[i][j].replace(" ", "")
            mol[i][j] = mol[i][j].replace(" ", "")
            IorO[i][j] = IorO[i][j].replace(" ", "")
            
            if besio[i][j].upper() == 'O':
                label_values[i].append(besio[i][j].upper())
            if mol[i][j].upper() == 'PRO':
                label_values[i].append(besio[i][j].upper()+'-'+mol[i][j].upper())
            if IorO[i][j].upper() == 'I' or IorO[i][j].upper() == 'O':
                #The below does not handle cases where BESIO or MOL has errors though...
                if mol[i][j].upper() != 'PRO':
                    label_values[i].append(besio[i][j].upper()+'-'+mol[i][j].upper()+'-'+IorO[i][j].upper())
                else: 
                    print("Weird. This Property is organic or inorganic? LOL")
            #PRIME OPPORTUNITY FOR ERROR HANDLING - IF ANYTHING NOT IN THE ABOVE CATS, SOMETHING IS WRONG
            
        i += 1   
    return all_tokens, label_values

In [423]:
print(labels[6])

['O', 'O', 'O', 'O', 'S-PRO', 'S-PRO', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-MOL-O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-MOL-I', 'O', 'S-MOL-O', 'O', 'O', 'S-MOL-O', 'B-MOL-O', 'E-MOL-O', 'B-MOL-O', 'E-MOL-O', 'O', 'B-MOL-O', 'E-MOL-O', 'O', 'O', 'O', 'O', 'B-PRO', 'E-PRO', 'O', 'O', 'S-PRO', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-MOL-I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-PRO', 'O', 'S-MOL-I', 'O', 'S-MOL-O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-PRO', 'O', 'S-PRO', 'O', 'O', 'O', 'O', 'S-MOL-O', 'S-PRO', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [419]:
tokens, labels = extract_xy_(df)

ioro
ioro
PRO
PRO
ioro
ioro
ioro
PRO
ioro
PRO
PRO
PRO
ioro
ioro
ioro
PRO
PRO
ioro
PRO
ioro
ioro
PRO
PRO
ioro
PRO
ioro
ioro
ioro
ioro
ioro
PRO
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
PRO
ioro
PRO
PRO
ioro
PRO
PRO
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
PRO
ioro
ioro
ioro
ioro
ioro
ioro
ioro
PRO
ioro
ioro
PRO
PRO
PRO
PRO
ioro
ioro
ioro
PRO
PRO
PRO
PRO
ioro
ioro
PRO
PRO
ioro
ioro
ioro
PRO
PRO
ioro
PRO
ioro
ioro
ioro
ioro
PRO
ioro
ioro
ioro
ioro
ioro
PRO
ioro
ioro
ioro
ioro
ioro
PRO
PRO
ioro
ioro
ioro
ioro
ioro
ioro
ioro
PRO
ioro
ioro
PRO
PRO
ioro
ioro
PRO
PRO
PRO
PRO
ioro
PRO
ioro
ioro
ioro
ioro
ioro
ioro
PRO
PRO
PRO
PRO
PRO
PRO
ioro
ioro
PRO
PRO
PRO
PRO
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
PRO
ioro
ioro
ioro
PRO
ioro
ioro
ioro
ioro
PRO
PRO
ioro
ioro
ioro
ioro
ioro
ioro
ioro
PRO
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
ioro
PRO
PRO
PRO
PRO
PRO
PRO
PRO
PRO
ioro
ioro
ioro
PRO
PRO
ioro
ioro
ioro
ioro


Ok, so now we have two lists. Both lists are structured so that each entry represents a unique paper (ie, tokens[1] is a whole paper). We need to now chop it down so that all the extra entries at the end are removed, and the list tokens[1] is only as long as there are words in that paper. 

Once we have done that for each entry, we will need to build a looping/wrapper function that will read every excel sheet in a directory. It'd be ideal if that looping function would append each new list to the previous list, so we could end up with every single labeled paper in a single set of two lists. 

After we have that function built, the next step is to try to regenerate our sentence-split structure. First step is to make each paper back into a single string. We'll do this by making a homemade inverse .split() function, which means we'll add each item in the tokens[1] list together with a single whitespace between them. Example is in the case ['The', 'dog', 'ran.'] we would want to regen the original string of ['The dog ran.']. We could do that by doing original_string += (token[1][i]+ ' '). Once we have made each paper in the list back into individual strings, we'll chop each string into individual sentences by using NLTK. 

All of the above is now done!

From that point, it's more standard BERT. We'll use BERT's tokenizer. We'll need to make sure we hand-extend each label to match the tokenization done by the BERT tokenizer so we don't have length mismatches (a la: https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/ function tokenize_and_preserve_labels) and then we'll send it to a dataloader. 

In [214]:
#Ok, need another def function that takes in a list of tokens, and a list of labels, and returns them each as a list of strings
def tokenized_to_string(token_list):
    token_stringlist = []
    for paper_tokens in token_list:
        paper_string = ""
        for i in paper_tokens:
            #This is basically an 'unsplit' method lol
            paper_string += (str(i) + " ")
        token_stringlist.append(paper_string)
    return token_stringlist

In [428]:
def labeled_sheets_to_listed_tokens(directory_url):
    """This function opens a directory of labeled excel sheets from David's excel sheets and returns the tokens as a list 
    of strings fully combined on a document level. It returns a list of strings, with each string being a full document."""
    files = os.listdir(directory_url)
    token_list = []
    label_list = []
    sent_labels = []
    for file in files:
        df = pd.read_excel(directory_url+file)
        token, label = extract_xy_(df)
        token_list += (tokenized_to_string(token))
        label_list += (label)
    #Now we tokenize each paper by sentences using NLTK:
    #we will also restructure labels to be ordered by sentences. 
    for i in range(len(token_list)):
        sentences = tokenize.sent_tokenize(token_list[i])
        token_list[i] = sentences
        short_term_labels = []
        for j in range(len(token_list[i])):    
            length = len(token_list[i][j].split())
            short_list = label_list[i][:length]
            short_term_labels.append(short_list)
            del(label_list[i][:length])
        sent_labels.append(short_term_labels)
    return token_list, sent_labels

In [475]:
list_o_tokens, list_o_labels = labeled_sheets_to_listed_tokens(dir_url)

Weird. This Property is organic or inorganic? LOL
Weird. This Property is organic or inorganic? LOL


In [472]:
print(list_o_tokens[5])
print(list_o_labels[5])

['In this work we study the carbonization resulting from an intumescence phenomenon of the fire-retardant formulation: ethylene terpolymer-ammonium polyphosphate/pentaerythritol.', 'Characterisation of the intumescent coating is carried out using infrared spectroscopy and MAS-NMR of the solid state.', 'The study shows the formation of organic phosphocarbonaceous esters, which limits depolymerisation or the evolution of gaseous hydrocarbons.', 'These esters prevent the development of condensed polyaromatic structures.', 'These latter are bridged by the polyethylenic links, which provide the mechanical properties of interest in the protective shield.', '©']
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-PRO', 'O', 'B-MOL-O', 'I-MOL-O', 'E-MOL-O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MOL-O', 'E-MOL-O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O',

In [230]:
from transformers import BertTokenizer

In [231]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

In [307]:
def max_encoded_length(token_list):
    max_len = 0
    for papers in token_list:
        for sentences in papers:
            input_ids = tokenizer.encode(sentences, add_special_characters = True)
            max_len = max(max_len, len(input_ids))
            
    return max_len

Ok, so the longer the dataset input, the worse performance is going to get. It turns out the performance of BERT scales quadratically with feature size. Esp. when we only have like 17 sentences greater than 150 words, it seems insane to use so much extra dimensionality. Instead, what we'll do is we'll chunk up sentences to a specific length. BUT, in order to get context, we need to have overlap of each chunked sequence, though it's not clear exactly how much overlap we will need, as this is kinda non-standard. My thought is if we limit max input length to 150, we could then chunk stuff up into 150-length inputs, wherein there are like 25 input overlaps on either side of the sentence.  

Alternatively, we could just delete any sentences that are greater than a certain size. To begin with, we will just delete sentences.

If we chop all sentences with greater than 150 words, we lose 1.4% of our sentences.
If we chop all sentences with greater than 160 words, 
we lose 1.24%
If we chop all with greater that 175, we lose 0.78%
If we chop with greater than 200, we lose 0.38% (5 sentences)


We'll use 150 at first, because we want to try to train locally if at all possible.

In [373]:
def delete_sentences(token_list, label_list, print_pop = False, max_length = 150):
    i = 0
    #lets use pop to remove these long sentences
    while i < len(token_list):
        list_of_j = []
        j = 0
        while j < len(token_list[i]):
            input_ids = tokenizer.encode(token_list[i][j], add_special_tokens = True)
            if len(input_ids) > 150:
                print("Found item length: " + str(len(input_ids)))
                list_of_j.append(j)
            j += 1
        k = len(list_of_j)-1
        #Gotta count backwards so we don't disturb the list structure
        while k > -1:
            if print_pop:
                print(token_list[i].pop(list_of_j[k]))
                print(label_list[i].pop(list_of_j[k]))
            else:
                token_list[i].pop(list_of_j[k])
                label_list[i].pop(list_of_j[k])
            k = k - 1
        i += 1 
    return token_list, label_list

In [476]:
short_sent_list, short_label_list = delete_sentences(list_o_tokens, list_o_labels)

Found item length: 181
Found item length: 189
Found item length: 172
Found item length: 270
Found item length: 178
Found item length: 191
Found item length: 171
Found item length: 151
Found item length: 235
Found item length: 161
Found item length: 509
Found item length: 174
Found item length: 207
Found item length: 248
Found item length: 180
Found item length: 155
Found item length: 165
Found item length: 173


In [457]:
max_encoded_length(short_sent_list)

149

Because of this, we'll set our maximum length inside of BERT to be 155, just to be safe. It's probably a bit unnecessary, but for our relatively small dataset, I bet the extra 6D won't be a crisis.

In [390]:
len(short_label_list[5][1])

17

In [379]:
len(short_sent_list[5])

6

Here's what I just learned: We can use the tensordataset loader things. We need to change our label list though. The label needs to have an O token for the CLS and SEP tokens, and that O token should also exist for the padding values. We also DO need to increase the length of each individual word label across the tokenized expansion of the word. 

Finally, once we've done all that, we need to convert the whole label list into individual integer mappings. We should build a label to number mapper, and then a number to label mapper at the same time. Should mostly just be a dictionary and key mapper type thing. 

In [436]:
#Long term we should absolutely consider getting rid of S, E labels
label_mapping = {'O': 0, "B-MOL-O": 1, "I-MOL-O": 2, "E-MOL-O": 3,
                "S-MOL-O": 4, "B-MOL-I": 5, "I-MOL-I": 6, "E-MOL-I": 7,
                "S-MOL-I": 8, "B-PRO": 9, "I-PRO": 10, "E-PRO": 11, "S-PRO": 12}


In [500]:
input_ids = []
attention_masks = []
label_shapes = []
count = 0

for abstracts, abst_labels in zip(short_sent_list, short_label_list):
    for sentences, sent_labels in zip(abstracts, abst_labels):
        encoded_dict = tokenizer.encode_plus(
                                    sentences,
                                    add_special_tokens = True,
                                    max_length = 155,
                                    pad_to_max_length = True,
                                    return_attention_mask = True,
                                    return_tensors = 'pt'
        )
        #Ok, now we get our labels based on encoded sizes. 
        #Make this a standalone function later instead of nesting
        #Need to start the CLS token to every label. 
        #This CLS token should be an 'O', to keep label
        #length matching consistent with the tokenized sentence
        extend_sent_labels = ['O']
        
        
        #I Bet the problem with things being read in is the difference in length of 
        #however this chunks sentences versus how the labels were originally split.
        
        for word, label in zip(sentences, sent_labels):
            tokenized_word = tokenizer.tokenize(word)
            #Find out how many chunks each word gets broken into
            n_subwords = len(tokenized_word)
            #Extend the length of the labels to match new word length
            #Put label in brackets so it knows you want n_subwords entries
            #of label, not label times n_subwords
            extend_sent_labels.extend([label]*n_subwords)

        #This handles increasing the length for padding and sep tokens
        #Go all the way to 155. Padding and  SEP should both be O
        extend_sent_labels.extend(['O']*(155-len(extend_sent_labels)))    
#
        #Next step, we need to use the dictionary lookup
        #to replace all the values from this list to become 
        #numbers. for loops to go through the whole list. 
        #print(sentences)
        #print(extend_sent_labels)
        for i in range(len(extend_sent_labels)):
            if extend_sent_labels[i] in label_mapping.keys():
                #Replace the label in extend_set_labels[i] from dict
                extend_sent_labels[i] = label_mapping[extend_sent_labels[i]]
        #print(extend_sent_labels)
        #Then, we make the labels list into a tensor.
        #extend_sent_labels = torch.tensor(extend_sent_labels)
        test_list = []
        test_list.append(extend_sent_labels)
        test_list = torch.tensor(test_list)
        #Build our attention mask, labels, and input ids of each item.
        #print(encoded_dict)
        label_shapes.append(test_list)
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        #print(encoded_dict['input_ids'])
#         count += 1
#         if count == 2:
#             print(label_shapes)
#             print(input_ids)
#             raise TypeError
print(len(label_shapes[1]))
print(len(input_ids[1]))
print(len(attention_masks[1]))

#Make lists we just built into tensors
input_ids = torch.cat(input_ids, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.cat(label_shapes, dim = 0)

print("Original Sentence: ", short_sent_list[0][0])
print("Tokenized IDs: ", input_ids[0])
print(labels[0])

1
1
1
Original Sentence:  In the interaction between gas molecules with single-walled carbon nanotube (SWCNT) we show that as a result of collisions the gas scattering contributes with an important background signal and should be considered in SWCNT-based gas sensors.
Tokenized IDs:  tensor([  101,  1999,  1996,  8290,  2090,  3806, 10737,  2007,  2309,  1011,
        17692,  6351, 28991, 28251,  2063,  1006, 25430,  2278,  3372,  1007,
         2057,  2265,  2008,  2004,  1037,  2765,  1997, 28820,  1996,  3806,
        17501, 16605,  2007,  2019,  2590,  4281,  4742,  1998,  2323,  2022,
         2641,  1999, 25430,  2278,  3372,  1011,  2241,  3806, 13907,  1012,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0

In [501]:
attention_masks.shape

torch.Size([1272, 155])

In [502]:
labels.shape

torch.Size([1272, 155])

In [503]:
input_ids.shape

torch.Size([1272, 155])

Ok. Now we have a label list, an attention mask, and an input id that are all the same shape, and contain all the data, in full integer form, so everything can be put into a DataLoader, and then we can start thinking about putting everything into the BERT model. Very exciting! Let's start with a dataloader.